# Data Processing

## Librairies imports

In [ ]:
# using Pkg;
# Pkg.add("CSV");
# Pkg.add("Random");
# Pkg.add("DataStructures");
# Pkg.add("BenchmarkTools");
# Pkg.add("DataFrames");
# Pkg.add("Statistics");
# Pkg.add("Dates");
# Pkg.add("Gadfly");
# Pkg.add("MLBase");
# Pkg.add("DecisionTree");
# Pkg.add("GLM");

In [ ]:
using CSV, DataFrames, GLM, Statistics, Dates, Gadfly, Random, MLBase, DecisionTree;
include("utils/precipitation.jl");

## Build features

### 1. Ouvrages_surverse.csv

#### ID_ouvrage, Latitude, Longitude, Height

In [ ]:
ouvrages = CSV.read("data/ouvrages-surverses.csv");
colnames = ["N_Env", "ID_SOMA", "ID_OUVRAGE", "NOM", "SOMA_SEC", "REGION", "TP_X", "TP_Y", "TP_Z", "TP_LAT", "TP_LNG", "EMI_X", "EMI_Y", "EMI_LNG", "EMI_LAT"];
names!(ouvrages, Symbol.(colnames));
# select!(ouvrages, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z]);

#### Replace missing Z index with mean

In [ ]:
ouvrages.TP_Z = coalesce.(ouvrages.TP_Z, mean(ouvrages[completecases(ouvrages), :].TP_Z));
first(shuffleDf(ouvrages), 5)

#### Visualtion des données chargées

In [ ]:
plot(ouvrages, x=:TP_Z, Geom.histogram(bincount=50), Guide.xlabel("Height of TropPlein"),Guide.ylabel("Frequency"))

In [ ]:
plot(ouvrages,x=:EMI_X, y=:EMI_Y, Geom.point)

In [ ]:
plot(ouvrages,x=:TP_X, y=:TP_Y, Geom.point)

### 2. Surverses.csv

#### NO_ouvrage, Date, Surverse

In [ ]:
surverses = CSV.read("data/surverses.csv", missingstring="-99999");

first(shuffleDf(surverses),5)

#### Filter months

In [ ]:
surverses = filter(row -> month(row.DATE) > 4, surverses);
surverses = filter(row -> month(row.DATE) < 11, surverses);

#### Filter non rain surverses

In [ ]:
raison = coalesce.(surverses[:,:RAISON],"Inconnue");
surverses[!,:RAISON] = raison;

surverses = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], surverses);
select!(surverses, [:NO_OUVRAGE, :DATE, :SURVERSE]);

#### Remove missing data and rename

In [ ]:
surverses = dropmissing(surverses, disallowmissing=true);
rename!(surverses, :NO_OUVRAGE => :ID_OUVRAGE);
first(shuffleDf(surverses), 5)

### 3. Precipitation.csv

#### Date, Heure, McTavish, Bellevue, Assomption, Trudeau, StHubert

#### Load and filter months between May & October included

In [ ]:
precipitations = CSV.read("data/precipitations.csv",missingstring="-99999");
rename!(precipitations, Symbol("St-Hubert")=>:StHubert);

precipitations = filter(row -> month(row.date) > 4, precipitations);
precipitations = filter(row -> month(row.date) < 11, precipitations); 

In [ ]:
first(shuffleDf(precipitations),5)

#### Replace missing data by 0

In [ ]:
#OLD WAY

#precipitation[!,:McTavish] = coalesce.(precipitation[:,:McTavish], 0);
#precipitation[!,:Bellevue] = coalesce.(precipitation[:,:Bellevue], 0);
#precipitation[!,:Assomption] = coalesce.(precipitation[:,:Assomption], 0);
#precipitation[!,:Trudeau] = coalesce.(precipitation[:,:Trudeau], 0);
#precipitation[!,:StHubert] = coalesce.(precipitation[:,:StHubert], 0);

#first(shuffleDf(precipitation), 5)

In [ ]:
precipitation_by_day = by(precipitations, :date,  
                            McTavish = :McTavish=>mean_wo_missing, 
                            Bellevue = :Bellevue=>mean_wo_missing, 
                            Assomption = :Assomption=>mean_wo_missing,
                            Trudeau = :Trudeau=>mean_wo_missing,
                            StHubert = :StHubert=>mean_wo_missing)

for i=1:size(precipitations,1)
    if isequal(precipitations[i, :McTavish], missing)
        precipitations[i,:McTavish] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:McTavish][1]
    end
    if isequal(precipitations[i, :Bellevue], missing)
        precipitations[i,:Bellevue] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Bellevue][1]
    end
    if isequal(precipitations[i, :Assomption], missing)
        precipitations[i,:Assomption] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Assomption][1]
    end
    if isequal(precipitations[i, :Trudeau], missing)
        precipitations[i,:Trudeau] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:Trudeau][1]
    end
    if isequal(precipitations[i, :StHubert], missing)
        precipitations[i,:StHubert] = filter(row-> row.date == precipitations[i,:date], precipitation_by_day)[!,:StHubert][1]
    end
end

first(shuffleDf(precipitations), 5)

###  Precipitation aggregation

#### Sum of precipitation for the day

In [ ]:
pcp_sum = by(precipitations, :date,  
            McTavish = :McTavish=>sum, 
            Bellevue = :Bellevue=>sum,
            Assomption = :Assomption=>sum, 
            Trudeau = :Trudeau=>sum, 
            StHubert = :StHubert=>sum);
first(shuffleDf(pcp_sum), 5)

#### Representation visuelle des données enregistrées des différentes stations. 
(C'est intéractif ! Vous pouvez choisir quelles distributions voir !)

In [ ]:
df_for_plot = pcp_sum
df_for_plot = filter(row -> year(row.date) == 2016, pcp_sum);
df_for_plot = melt(df_for_plot, :date)

plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2019, pcp_sum);
df_for_plot = melt(df_for_plot, :date)

plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

In [ ]:
date_to_plot = 2013;
df_for_plot = filter(row -> year(row.date) == date_to_plot, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plt_a = plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable);

surv_to_plot = filter(row -> year(row.DATE) == date_to_plot && row.SURVERSE == 1, surverses)
select!(surv_to_plot, [:DATE, :SURVERSE])
surv_date = by(surv_to_plot, :DATE, :SURVERSE => sum)
plt_b = plot(surv_date, x=:DATE, y=:SURVERSE_sum, Geom.line);
set_default_plot_size(25cm, 13cm)
hstack(plt_a, plt_b)

In [ ]:
# first(shuffleDf(surverses), 10);
surv_to_plot = filter(row -> year(row.DATE) == 2014 && row.SURVERSE == 1, surverses)
select!(surv_to_plot, [:DATE, :SURVERSE])
surv_date = by(surv_to_plot, :DATE, :SURVERSE => sum)
plot(surv_date, x=:DATE, y=:SURVERSE_sum, Geom.line)

#### Maximum precipitation in an hour for the day

In [ ]:
pcp_max = by(precipitations, :date,  
            McTavish = :McTavish=>maximum,
            Bellevue = :Bellevue=>maximum, 
            Assomption = :Assomption=>maximum,
            Trudeau = :Trudeau=>maximum,
            StHubert = :StHubert=>maximum)
first(shuffleDf(pcp_max),5)

#### Representation visuelle des données enregistrées des différentes stations

(C'est intéractif ! Vous pouvez choisir quelles distributions voir !)

In [ ]:
df_for_plot = pcp_max
df_for_plot = filter(row -> year(row.date) == 2013, pcp_max);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

In [ ]:
df_for_plot = filter(row -> year(row.date) == 2019, pcp_max);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

#### Maximum precipitation during three consecutive hours in a day

In [ ]:
pcp_max3h = by(precipitations, :date,
                McTavish = :McTavish=>maximum3,
                Bellevue = :Bellevue=>maximum3,
                Assomption = :Assomption=>maximum3,
                Trudeau = :Trudeau=>maximum3,
                StHubert = :StHubert=>maximum3)
first(shuffleDf(pcp_max3h),5)

#### Representation visuelle des données enregistrées des différentes stations

(C'est intéractif ! Vous pouvez choisir quelles distributions voir !)

In [ ]:
df_for_plot = pcp_max3h
df_for_plot = filter(row -> year(row.date) == 2018, pcp_sum);
df_for_plot = melt(df_for_plot, :date)
plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

#### Visualisation of all three agregations for 1 meteo station

In [ ]:
date_to_pt = 2018;
loc = :McTavish
mct_sum = pcp_sum[:,[1,2]]
rename!(mct_sum, loc => :Sum);
mct_sum = filter(row -> year(row.date) == date_to_pt, mct_sum);

mct_max = pcp_max[:,[1,2]]
rename!(mct_max,loc => :Max);
mct_max = filter(row -> year(row.date) == date_to_pt, mct_max);

mct_max3h = pcp_max3h[:,[1,2]]
rename!(mct_max3h,loc => :Max3h);
mct_max3h = filter(row -> year(row.date) == date_to_pt, mct_max3h);

df_for_plot = join(mct_sum, mct_max3h, on = :date);
df_for_plot = join(df_for_plot, mct_max, on = :date);
df_for_plot = melt(df_for_plot, :date)


plot(df_for_plot, x=:date, y=:value, Geom.line, color=:variable)

### Join Dataframes

In [ ]:
X_test = CSV.read("data/test.csv");
rename!(X_test, :NO_OUVRAGE => :ID_OUVRAGE);

In [ ]:
X_test = join(ouvrages, X_test, on =:ID_OUVRAGE);
features = join(ouvrages, surverses, on =:ID_OUVRAGE);
srv = filter(row -> row.SURVERSE == 1, features);
first(shuffleDf(srv), 10)

In [ ]:
# df_for_geo_plot = filter(row -> row.DATE == Date(2018,7,25), features)
df_for_geo_plot = filter(row -> row.DATE == Date(2015, 5, 30), features)
# df_for_geo_plot = filter(row -> row.DATE == Date(2017, 10, 9), features)
df_for_geo_plot[:SURVERSE] = convert(Array{Bool,1}, df_for_geo_plot[:SURVERSE])
plot(df_for_geo_plot, x=:TP_LNG, y=:TP_LAT, Geom.point, color=:SURVERSE, Guide.title("2018-07-25, état des surverses"))
#first(df_for_geo_plot,5) 

### Add precipitation data to features

#### Get stations lat-lng

In [ ]:
station_df = DataFrame(STATION = String[], LAT = Float64[], LNG = Float64[]);

push!(station_df, ["McTavish", 45.504742, -73.579167]);
push!(station_df, ["Bellevue", 45.427222, -73.929167]);
push!(station_df, ["Assomption", 45.809444, -73.434722]);
push!(station_df, ["Trudeau", 45.467778, -73.741667]);
push!(station_df, ["StHubert", 45.5175, -73.416944]);

station_df

### Augment Features

#### Add pcp_sum, pcp_max, pcp_max3 columns

In [ ]:
function addColsForPrecipationPerDay(df)
    df[!, :PCP_SUM] = zeros(size(df, 1));
    df[!, :PCP_MAX] = zeros(size(df, 1));
    df[!, :PCP_MAX3] = zeros(size(df, 1));
    df[!, :METEO] = fill("", size(df, 1));
    return df
end

X_test = addColsForPrecipationPerDay(X_test)
permutecols!(X_test, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :METEO, :PCP_SUM, :PCP_MAX, :PCP_MAX3]);
    
features = addColsForPrecipationPerDay(features)
permutecols!(features, [:ID_OUVRAGE, :TP_LAT, :TP_LNG, :TP_Z, :DATE, :METEO, :PCP_SUM, :PCP_MAX, :PCP_MAX3, :SURVERSE]);

first(shuffleDf(features), 5)

#### Find closest station to each ouvrage and add pcp_sum and pcp_max to it

In [ ]:
function fillPrecipitationWithClosestMeteoStation(df)
    for i=1:size(df, 1)
        id_ouvrage = df[i, 1]; 
        closest_station = "McTavish"; # initial value
        shortest_dist = -1;

        # Find closest station
        for j=1:size(station_df, 1)
            dist = findDistance(df[i, :TP_LAT], df[i, :TP_LNG], station_df[j, :LAT], station_df[j, :LNG]);

            if shortest_dist == -1 || dist < shortest_dist
                shortest_dist = dist;
                closest_station = station_df[j, :STATION];
            end
        end

        # Augment comb with a weighted p_sum, based on the distance to the station
        p_sum = pcp_sum[∈([df[i, :DATE]]).(pcp_sum.date), Symbol(closest_station)];
    #     comb[i, :PCP_SUM] = p_sum[1] * (1 - shortest_dist);
        df[i, :PCP_SUM] = p_sum[1]; 

        # Augment comb with a weighted p_max, based on the distance to the station
        p_max = pcp_max[∈([df[i, :DATE]]).(pcp_max.date), Symbol(closest_station)]
    #     comb[i, :PCP_MAX] = p_max[1] * (1 - shortest_dist);
        df[i, :PCP_MAX] = p_max[1];

        # Augment comb with a weighted p_max3h, based on the distance to the station
        p_max3 = pcp_max3h[∈([df[i, :DATE]]).(pcp_max3h.date), Symbol(closest_station)]
    #     comb[i, :PCP_MAX3] = p_max3[1] * (1 - shortest_dist);
        df[i, :PCP_MAX3] = p_max3[1]; 

        df[i, :METEO] = closest_station
    end
    return df
end

In [ ]:
X_test = fillPrecipitationWithClosestMeteoStation(X_test)
features = fillPrecipitationWithClosestMeteoStation(features)
first(shuffleDf(features), 5)

#### Remove outlier in PCP_SUM and PCP_MAX AND PCP_MAX3 that cause compression

In [ ]:
X_test[X_test[:PCP_SUM] .> 750, :PCP_SUM] = 750;
X_test[X_test[:PCP_MAX] .> 500, :PCP_MAX] = 500;
X_test[X_test[:PCP_MAX3] .> 750, :PCP_MAX3] = 750;

features[features[:PCP_SUM] .> 750, :PCP_SUM] = 750;
features[features[:PCP_MAX] .> 500, :PCP_MAX] = 500;
features[features[:PCP_MAX3] .> 750, :PCP_MAX3] = 750;

# first(shuffleDf(filter(row -> row.SURVERSE == 1, features)), 5)

#### TP location colored by their affiliation to its closest meteo station

In [ ]:
df_for_geo_plot = filter(row -> row.DATE == Date(2018,7,25), features)
df_for_geo_plot[:SURVERSE] = convert(Array{Bool,1}, df_for_geo_plot[:SURVERSE])
plot(df_for_geo_plot, x=:TP_LNG, y=:TP_LAT, Geom.point, color=:METEO, Guide.title("2018-07-25, Regroupement par station météo"))
#first(df_for_geo_plot,5) 

#### Ouvrage 3350-07D for 2018, when it overflow and quantity based on max3h rain fallen.

In [ ]:

# 3260-01D dans Rivière-des-Prairies
# 3350-07D dans Ahunstic
# 4240-01D dans Pointe-aux-Trembles
# 4350-01D dans le Vieux-Montréal
# 4380-01D dans Verdun

id_ouvrage_to_show = "3350-07D"
df_temp = filter(row -> row.ID_OUVRAGE ∈ [id_ouvrage_to_show], features)
df_temp = filter(row -> year(row.DATE) == 2018, df_temp);
df_temp = df_temp[!,[:ID_OUVRAGE, :DATE, :PCP_MAX3, :SURVERSE]]
df_temp[:SURVERSE] = convert(Array{Bool,1}, df_temp[:SURVERSE])



plot(df_temp, x=:DATE, y=:PCP_MAX3, Geom.point, color=:SURVERSE,Guide.title(id_ouvrage_to_show))
#first(shuffleDf(df_temp), 10)

### Split dates into months and days

In [ ]:
function splitDateByMonthAndDay(df)
    df[!,:MONTH] = month.(df.DATE);
    df[!,:DAY] = day.(df.DATE);
    return df
end

X_test = splitDateByMonthAndDay(X_test)
features = splitDateByMonthAndDay(features)
first(shuffleDf(features[!, [:DATE, :MONTH, :DAY]]), 5)

## Split Dataframes into train, validation and test

In [ ]:
function partitionTrainTest(data, at = 0.8) # https://discourse.julialang.org/t/simple-tool-for-train-test-split/473/2
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    return data[train_idx,:], data[test_idx,:]
end


In [ ]:
function standarizeTrainTestCol(X_train, X_test, col)
    mean_train_col = mean(X_train[!, col]);
    std_train_col = std(X_train[!, col]);
    X_train[!, col] = (X_train[!, col] .- mean_train_col) ./ std_train_col;
    X_test[!, col] = (X_test[!, col] .- mean_train_col) ./ std_train_col;
    
    return X_train, X_test
end
    

####  Standardize the PCP and Date before splitting into X_train and X_val

In [ ]:
filter(row -> year(row.DATE) == 2019, features)

In [ ]:
X_train = filter(row-> year(row.DATE) .< 2019, features)
X_train = features

X_train, X_test = standarizeTrainTestCol(X_train, X_test, :TP_LAT)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :TP_LNG)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :TP_Z)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :PCP_SUM)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :PCP_MAX)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :PCP_MAX3)
X_train, X_test = standarizeTrainTestCol(X_train, X_test, :MONTH)
X_train_all, X_test = standarizeTrainTestCol(X_train, X_test, :DAY)

X_train, X_val = partitionTrainTest(X_train_all, 0.8)

first(shuffleDf(X_train),5)

In [ ]:
first(shuffleDf(X_test),5)

In [ ]:
first(shuffleDf(X_val),5)

###  Standardize the PCP and Date

In [ ]:
# mean_pcpsum = mean(features.PCP_SUM);
# std_pcpsum = std(features.PCP_SUM);
# features.PCP_SUM = (features.PCP_SUM .- mean_pcpsum) ./ std_pcpsum;

# mean_pcpmax = mean(features.PCP_MAX);
# std_pcpmax = std(features.PCP_MAX);
# features.PCP_MAX = (features.PCP_MAX .- mean_pcpmax) ./ std_pcpmax;

# mean_pcpmax3 = mean(features.PCP_MAX3);
# std_pcpmax3 = std(features.PCP_MAX3);
# features.PCP_MAX3 = (features.PCP_MAX3 .- mean_pcpmax3) ./ std_pcpmax3;

# meanmonth = mean(features.MONTH);
# stdmonth = std(features.MONTH);
# features.MONTH = (features.MONTH .- meanmonth) ./ stdmonth;

# meanday = mean(features.DAY);
# stdday = std(features.DAY);
# features.DAY = (features.DAY .- meanday) ./ stdday;

In [ ]:
first(shuffleDf(filter(row -> row.SURVERSE == 1, features)), 5)

# Logistic Regression

In [ ]:
# categorical!(X_train, :METEO)
# levels(X_train[:, :METEO])

In [ ]:
names_glm = [:TP_LAT, :TP_LNG, :TP_Z, :MONTH, :DAY, :PCP_SUM, :METEO, :SURVERSE];
names(X_train)

In [ ]:
val_form = @formula(SURVERSE ~ TP_LAT + TP_LNG + TP_Z + MONTH + DAY + PCP_SUM + METEO);
val_model = glm(val_form, X_train, Bernoulli(), LogitLink())

In [ ]:
val_labels = X_val[!, :SURVERSE];
val_pred = GLM.predict(val_model, X_val);

threshold = 0.07;
val_pred[val_pred .>= threshold] .= 1.0;
val_pred[val_pred .< threshold] .= 0.0;
val_pred = convert(Array{Int}, trunc.(val_pred));

r = roc(val_labels, val_pred);
f1score(r)

In [ ]:
names_glm = [:SURVERSE, :TP_LNG, :TP_Z, :MONTH, :PCP_SUM, :METEO];
val_form = @formula(SURVERSE ~ TP_LNG + TP_Z + MONTH + PCP_SUM + METEO);
names(X_train)

In [ ]:
niter = 20;
batch_score = 0;
batch_threshold = 0;

for i=1:niter
    # Split train and val sets
    X_train_glm, X_val_glm = partitionTrainTest(X_train_all, 0.8);
    
    # Build features and labels
    train_features = X_train_glm[:, names_glm];
    
    # Build model
    val_model = glm(val_form, train_features, Bernoulli(), LogitLink())
    
    # Validate model
    val_features = X_val_glm[:, names_glm];
    val_labels = X_val_glm[!, :SURVERSE];
    
    # Get best threshold
    start_threshold = 0.090;
    max_threshold = 0.095;
    step = 0.0002;
    
    best_threshold = start_threshold;
    score = -1;
    
    # Get best threshold
    for j=start_threshold:step:max_threshold
        val_pred = GLM.predict(val_model, val_features);
        val_pred[val_pred .>= j] .= 1.0;
        val_pred[val_pred .< j] .= 0.0;
        val_pred = convert(Array{Int}, trunc.(val_pred))
        
        r = roc(val_labels, val_pred);
        new_score = f1score(r);
        
        if new_score > score
            score = new_score
            best_threshold = j
        end
    end
    
    batch_score += score;
    batch_threshold += best_threshold;
end

batch_threshold = batch_threshold / niter;
batch_score = batch_score / niter

In [ ]:
batch_threshold

#### Full train

In [ ]:
submit_features = X_train_all[:, names_glm];
submit_labels = X_train_all[:, :SURVERSE];

In [ ]:
model = glm(val_form, submit_features, Bernoulli(), LogitLink())

In [ ]:
categorical!(X_test, :METEO)
levels(X_test[:, :METEO])

In [ ]:
test_features = X_test[:, setdiff(names_glm, [:SURVERSE])]
test_labels = GLM.predict(val_model, test_features);
test_labels[test_labels .>= batch_threshold] .= 1.0;
test_labels[test_labels .< batch_threshold] .= 0.0;
test_labels = convert(Array{Int}, trunc.(test_labels))

### Submission

# Random Forest

In [ ]:
X_train_rf = X_train;
X_val_rf = X_val;
X_test_rf = X_test;
X_train_all_rf = X_train_all;

In [ ]:
X_train_rf[:, :METEO] = broadcast(convert_meteo, X_train[:, :METEO]);
X_val_rf[:, :METEO] = broadcast(convert_meteo, X_val[:, :METEO]);
X_test_rf[:, :METEO] = broadcast(convert_meteo, X_test[:, :METEO]);
X_train_all_rf[:, :METEO] = broadcast(convert_meteo, X_train_all[:, :METEO]);

In [ ]:
names_ft = [:TP_LAT, :TP_LNG, :TP_Z, :MONTH, :METEO, :PCP_SUM, :PCP_MAX, :PCP_MAX3];
nft = 5;
ntrees = 120;
podata = 0.85;
maxd = 28;
train_features = convert(Matrix{Float64},X_train_rf[:, names_ft]);
train_labels = X_train_rf[:, :SURVERSE];

#### Single validation

In [ ]:
val_model = build_forest(train_labels, train_features, nft, ntrees, podata, maxd)

In [ ]:
val_model = build_forest(train_labels, train_features, nft, ntrees, podata, maxd)
val_features = convert(Matrix{Float64},X_val_rf[:, names_ft]);
val_labels = X_val_rf[!, :SURVERSE];
val_pred = apply_forest(val_model, val_features);

r = roc(val_labels, val_pred);
f1score(r)

#### Batch validation

In [ ]:
niter = 10;
batch_score = 0;

for i=1:niter
    # Split train and val sets
    X_train_rf, X_val_rf = partitionTrainTest(X_train_all_rf, 0.8);
    
    # Build features and labels
    train_features = convert(Matrix{Float64},X_train_rf[:, names_ft]);
    train_labels = X_train_rf[:, :SURVERSE];
    
    # Build model
    val_model = build_forest(train_labels, train_features, nft, ntrees, podata, maxd);
    
    # Validate model
    val_features = convert(Matrix{Float64},X_val_rf[:, names_ft]);
    val_labels = X_val_rf[!, :SURVERSE];
    val_pred = apply_forest(val_model, val_features);

    r = roc(val_labels, val_pred);
    batch_score += f1score(r);
end

batch_score = batch_score / niter

#### Train on all data

In [ ]:
submit_features =  convert(Matrix{Float64}, X_train_all_rf[:, names_ft]);
submit_labels = X_train_all_rf[:, :SURVERSE];
model = build_forest(submit_labels, submit_features, nft, ntrees, podata, maxd)

test_features = convert(Matrix{Float64},X_test_rf[:, names_ft]);
test_labels = apply_forest(model, test_features)

In [ ]:
ID = X_test_rf[:,:ID_OUVRAGE].*"_".*string.(X_test_rf[:,:DATE])
sampleSubmission = DataFrame(ID = ID, Surverse=test_labels)
CSV.write("submissions/mc-submission-16.csv",sampleSubmission)